In [24]:
import pandas as pd
import numpy as np

# Not normalized

In [74]:
ratings = pd.read_csv('assign_spreadsheet.csv', index_col=0)
corr = ratings.corr()

In [96]:
def get_top_5_neighbors(corr, user):
    corr_user = corr.loc[user].sort_values()[-6:-1]
    return (list(corr_user.index), corr_user.tolist())

def get_top_3_movies(ratings, corr, user):
    users_neighbors, correlations = get_top_5_neighbors(corr, user)

    ratings_user = dict()
    for movie, movie_ratings in ratings.loc[:, users_neighbors].fillna(-1).iterrows():
        total_weights = 0
        total_rating = 0
        for rating, corr_user in zip(movie_ratings, correlations):
            if rating >= 0:
                total_weights += corr_user
                total_rating += rating * corr_user
        ratings_user[movie] = total_rating / total_weights if total_weights > 0 else None
    return pd.Series(ratings_user).sort_values(ascending = False)[:3]

In [97]:
get_top_3_movies(ratings, corr, '3867').round(3)

1891: Star Wars: Episode V - The Empire Strikes Back (1980)    4.760
155: The Dark Knight (2008)                                    4.551
122: The Lord of the Rings: The Return of the King (2003)      4.508
dtype: float64

In [98]:
get_top_3_movies(ratings, corr, '89').round(3)

238: The Godfather (1972)               4.894
278: The Shawshank Redemption (1994)    4.882
807: Seven (a.k.a. Se7en) (1995)        4.774
dtype: float64

# Normalized

In [94]:
def get_top_3_movies_normalized(ratings, corr, user):
    users_neighbors, correlations = get_top_5_neighbors(corr, user)
    r_u = ratings.loc[:, user].mean()
    r_ns = ratings.loc[:, users_neighbors].mean()

    ratings_user = dict()
    for movie, movie_ratings in ratings.loc[:, users_neighbors].fillna(-1).iterrows():
        total_weights = 0
        total_rating = 0
        for rating, corr_user, r_n in zip(movie_ratings, correlations, r_ns):
            if rating >= 0:
                total_weights += corr_user
                total_rating += (rating - r_n) * corr_user
        ratings_user[movie] = r_u + total_rating / total_weights if total_weights > 0 else None
    return pd.Series(ratings_user).sort_values(ascending = False)[:3]

In [101]:
get_top_3_movies_normalized(ratings, corr, '3867').round(3)

1891: Star Wars: Episode V - The Empire Strikes Back (1980)    5.246
155: The Dark Knight (2008)                                    4.857
77: Memento (2000)                                             4.778
dtype: float64

In [100]:
get_top_3_movies_normalized(ratings, corr, '89').round(3)

238: The Godfather (1972)               5.322
278: The Shawshank Redemption (1994)    5.261
275: Fargo (1996)                       5.241
dtype: float64